In [1]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
#ws = Workspace.get(name='quick-starts-ws-137411')
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-139053
Azure region: southcentralus
Subscription id: 6b4af8be-9931-443e-90f6-c4c34a1f9737
Resource group: aml-quickstarts-139053


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name = 'my-compute'
try:
    cpu_cluster = ComputeTarget(workspace=ws, name= compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, compute_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)



Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler

### YOUR CODE HERE ###
ps = RandomParameterSampling({
    "--C": choice (1.0,2.0,3.0,4.0),
    "--max_iter":choice (50,75,100,125,150)
})


# Specify a Policy
policy = BanditPolicy (evaluation_interval=1, slack_factor=0.2)

if 'training' not in os.listdir():
    os.mkdir('./training')
    
import shutil
shutil.copy('train.py','./training')

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './training',
            compute_target= cpu_cluster,
             entry_script = 'train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,hyperparameter_sampling=ps,
                                     policy= policy,
                                     primary_metric_name = "Accuracy",
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 20,
                                     max_concurrent_runs=3)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hd_run = exp.submit(hyperdrive_config)


In [5]:
RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print ('Best Run Id: ' , best_run.id)
print('Accuracy: ', best_run_metrics ['Accuracy'])
print('Regularization Strength: ', best_run_metrics ['Regularization Strength:'])
print('Max Iterations: ', best_run_metrics ['Max iterations:'])


model = best_run.register_model (
                                 model_name= 'project1model',
                                 model_path = './',
                                 properties = {'Accuracy': best_run_metrics ['Accuracy'],
                                            'Regularization Strength': best_run_metrics ['Regularization Strength:'],
                                            'Max Iterations': best_run_metrics ['Max iterations:']})

Best Run Id:  HD_b6af50c8-2aeb-469c-9bfb-36a70dc2089b_1
Accuracy:  0.9128195696155968
Regularization Strength:  4.0
Max Iterations:  75


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
dset = TabularDatasetFactory.from_delimited_files(data)

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dset)

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dset,
    compute_target = cpu_cluster,
    label_column_name='y',
    n_cross_validations=5)

In [11]:
# Submit your automl run
from azureml.core import Workspace, Experiment
### YOUR CODE HERE ###
ws = Workspace.from_config()
AML_experiment = Experiment(ws, "AML_exp")
AML_run = AML_experiment.submit(automl_config,show_output = True)
RunDetails(AML_run).show()

Running on remote.
No run_configuration provided, running on my-compute with default configuration
Running on remote compute: my-compute
Parent Run ID: AutoML_9db2f18c-609e-42d7-8fce-fd341df9e7b1

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a false

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [12]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run,fitted_model= AML_run.get_output()

print(best_run)
print(fitted_model)

best_run_metrics = best_run.get_metrics()
for name in best_run_metrics:
    metric = best_run_metrics[name]
    print(name,metric)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: AML_exp,
Id: AutoML_9db2f18c-609e-42d7-8fce-fd341df9e7b1_21,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_leaf=0.01,
                                                                                                    min_samples_split=0.2442105263157895,
          

In [13]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":1,"CurrentNodeCount":1}',
 'ensembled_iterations': '[1, 0, 14, 11, 15, 5]',
 'ensembled_algorithms': "['XGBoostClassifier', 'LightGBM', 'XGBoostClassifier', 'XGBoostClassifier', 'LightGBM', 'RandomForest']",
 'ensemble_weights': '[0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]',
 'best_individual_pipeline_score': '0.9149317147192717',
 'best_individual_iteration': '1',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}